In [ ]:
!pip install nba_api
from nba_api.stats.endpoints import leaguedashplayerstats
from nba_api.stats.endpoints import leaguestandings
import pandas as pd
import numpy as np

def get_player_stats(season) :
  df = leaguedashplayerstats.LeagueDashPlayerStats(
      season = season,
      season_type_all_star='Regular Season',
      per_mode_detailed='PerGame'
  )
  stats = df.get_data_frames()[0]
  stats = stats.rename(columns = {'PLAYER_NAME' : 'PLAYER'})
  stats['PLAYER'] = stats['PLAYER'].replace('Cam Reynolds', 'Cameron Reynolds')
  stats['PLAYER'] = stats['PLAYER'].replace('Trevon Scott', 'Tre Scott')
  stats['PLAYER'] = stats['PLAYER'].replace('Ruben Nembhard Jr.', 'RJ Nembhard')
  stats['PLAYER'] = stats['PLAYER'].replace('Nate Williams', 'Jeenathan Williams')
  stats['PLAYER'] = stats['PLAYER'].replace('Matt Hurt', 'Matthew Hurt')
  stats['PLAYER'] = stats['PLAYER'].replace('Ronald Holland II', 'Ron Holland')
  return stats[['PLAYER_ID', 'PLAYER', 'TEAM_ID', 'GP', 'W_PCT', 'FGA', 'FG_PCT', 'FG3A', 'FG3_PCT', 'PTS', 'REB', 'AST', 'TOV', 'STL', 'BLK']].sort_values(by='PTS', ascending=False)

In [ ]:
player_stats = get_player_stats('2024-25')
player_stats.head(20)

,PLAYER_ID,PLAYER,TEAM_ID,GP,W_PCT,FGA,FG_PCT,FG3A,FG3_PCT,PTS,REB,AST,TOV,STL,BLK
490,1628983,Shai Gilgeous-Alexander,1610612760,76,0.829,21.8,0.519,5.7,0.375,32.7,5.0,6.4,2.4,1.7,1.0
180,203507,Giannis Antetokounmpo,1610612749,67,0.597,19.7,0.601,0.9,0.222,30.4,11.9,6.5,3.1,0.9,1.2
423,203999,Nikola Jokić,1610612743,70,0.657,19.5,0.576,4.7,0.417,29.6,12.7,10.2,3.3,1.8,0.6
366,1629029,Luka Dončić,1610612747,50,0.620,20.5,0.450,9.6,0.368,28.2,8.2,7.7,3.6,1.8,0.4
29,1630162,Anthony Edwards,1610612750,79,0.608,20.4,0.447,10.3,0.395,27.6,5.7,4.5,3.2,1.2,0.6
263,1628369,Jayson Tatum,1610612738,72,0.736,20.3,0.452,10.1,0.343,26.8,8.7,6.0,2.9,1.1,0.5
330,201142,Kevin Durant,1610612756,62,0.532,18.1,0.527,6.0,0.430,26.6,6.0,4.2,3.1,0.8,1.2
545,1630178,Tyrese Maxey,1610612755,52,0.385,21.0,0.437,9.2,0.337,26.3,3.3,6.1,2.4,1.8,0.4
69,1630595,Cade Cunningham,1610612765,70,0.543,20.8,0.469,6.0,0.356,26.1,6.1,9.1,4.4,1.0,0.8
226,1628973,Jalen Brunson,1610612752,65,0.615,18.5,0.488,6.1,0.383,26.0,2.9,7.3,2.5,0.9,0.1


In [ ]:
import requests
from bs4 import BeautifulSoup

def get_offensive_stats(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_advanced.html"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'id': 'advanced'})
    df = pd.read_html(str(table))[0]
    df = df[df['Player'] != 'Player']
    df = df[~df.duplicated(subset=['Player'], keep='first')]
    df.columns = df.columns.str.replace(' ', '')
    df.columns = df.columns.str.upper()
    columns_to_keep = ['PLAYER', 'TS%', 'USG%','OWS', 'OBPM']
    return df[columns_to_keep].reset_index(drop=True)

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports/universe amd64 Packages [35.2 kB]
Fetched 290 kB in 1s (229 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy 

In [ ]:
offensive_2021 = get_offensive_stats(2021)
offensive_2021.head(20)

/tmp/ipython-input-3-701077757.py:38: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


,PLAYER,TS%,USG%,OWS,OBPM
0,Julius Randle,.567,29.3,3.4,2.9
1,RJ Barrett,.535,23.4,0.9,-0.9
2,Nikola Jokić,.647,29.6,12.2,9.1
3,Buddy Hield,.567,20.7,1.5,1.0
4,Damian Lillard,.623,31.4,9.6,7.5
5,Terry Rozier,.575,24.4,3.3,2.2
6,Russell Westbrook,.509,30.2,0.5,2.6
7,Andrew Wiggins,.568,23.3,1.2,0.0
8,Mikal Bridges,.667,14.9,5.4,2.4
9,Nikola Vučević,.560,29.3,3.3,4.9


In [ ]:
def get_defensive_stats(season) :
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_advanced.html"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'id': 'advanced'})
    df = pd.read_html(str(table))[0]
    df = df[df['Player'] != 'Player']
    df = df[~df.duplicated(subset=['Player'], keep='first')]
    df.columns = df.columns.str.replace(' ', '')
    df.columns = df.columns.str.upper()
    columns_to_keep = ['PLAYER', 'DWS', 'DBPM']
    return df[columns_to_keep].reset_index(drop=True)

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
defensive_2021 = get_defensive_stats(2021)
defensive_2021.head(20)

/tmp/ipython-input-5-1432255247.py:37: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


,PLAYER,DWS,DBPM
0,Julius Randle,4.3,0.9
1,RJ Barrett,3.1,-0.3
2,Nikola Jokić,3.4,3.0
3,Buddy Hield,0.7,-1.6
4,Damian Lillard,0.8,-1.3
5,Terry Rozier,2.0,-0.5
6,Russell Westbrook,3.2,1.1
7,Andrew Wiggins,2.7,-0.4
8,Mikal Bridges,2.6,0.9
9,Nikola Vučević,3.0,0.4


In [ ]:
def get_advanced_stats(season) :
  url = f"https://www.basketball-reference.com/leagues/NBA_{season}_advanced.html"
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')
  table = soup.find('table', {'id': 'advanced'})
  df = pd.read_html(str(table))[0]
  df = df[df['Player'] != 'Player']
  df = df[~df.duplicated(subset=['Player'], keep='first')]
  df.columns = df.columns.str.replace(' ', '')
  df.columns = df.columns.str.upper()
  columns_to_keep = ['PLAYER', 'PER', 'VORP', 'AWARDS']
  return df[columns_to_keep].reset_index(drop=True)

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
advanced_2021 = get_advanced_stats(2021)
advanced_2021.head(10)

/tmp/ipython-input-10-2253921246.py:36: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


,PLAYER,PER,VORP,AWARDS
0,Julius Randle,19.7,3.9,"MVP-8,MIP-1,AS,NBA2"
1,RJ Barrett,13.4,0.5,NaN
2,Nikola Jokić,31.3,8.8,"MVP-1,AS,NBA1"
3,Buddy Hield,12.8,0.9,NaN
4,Damian Lillard,25.6,5.0,"MVP-7,AS,NBA2"
5,Terry Rozier,17.1,2.2,NaN
6,Russell Westbrook,19.5,3.4,MVP-11
7,Andrew Wiggins,15.0,1.0,NaN
8,Mikal Bridges,16.4,3.1,NaN
9,Nikola Vučević,22.9,4.4,AS


In [ ]:
def get_team_record(season) :
  standings = leaguestandings.LeagueStandings(season=season).get_data_frames()[0]
  return standings[['TeamID', 'TeamName', 'Record']]

In [ ]:
get_team_record('2020-21')

,TeamID,TeamName,Record
0,1610612755,76ers,49-23
1,1610612762,Jazz,52-20
2,1610612751,Nets,48-24
3,1610612756,Suns,51-21
4,1610612743,Nuggets,47-25
5,1610612749,Bucks,46-26
6,1610612746,Clippers,47-25
7,1610612752,Knicks,41-31
8,1610612737,Hawks,41-31
9,1610612742,Mavericks,42-30


In [ ]:
import unicodedata
import re

def normalize_name(name):
    if pd.isna(name):
        return ''
    # Convert to lowercase
    name = name.lower()
    # Remove accents (e.g., 'Felício' → 'Felicio')
    name = unicodedata.normalize('NFKD', name).encode('ASCII', 'ignore').decode('utf-8')
    # Remove suffixes (Jr., II, III, IV)
    name = re.sub(r'\s(jr\.?|sr\.?|ii|iii|iv)$', '', name)
    # Remove punctuation and extra spaces
    name = re.sub(r'[^a-z\s]', '', name)
    name = ' '.join(name.split())  # remove extra whitespace
    return name
def capitalize_name(name) :
  parts = name.split()
  if len(parts) >= 2 :
    return parts[0][0].upper() + parts[0][1:] + ' ' + parts[1][0].upper() + parts[1][1:]
  else :
    return name[0].upper() + name[1:]
def is_all_nba(awards) :
  if pd.isna(awards) :
    return np.nan
  if 'NBA1' in awards :
    return 1
  elif 'NBA2' in awards :
    return 2
  elif 'NBA3' in awards :
    return 3
  else :
    return np.nan
def create_final_data(season):
  end_season = '20' + season.split('-')[1]
  player_stats = get_player_stats(season)
  player_stats['PLAYER'] = player_stats['PLAYER'].apply(normalize_name)
  offensive_stats = get_offensive_stats(end_season)
  offensive_stats['PLAYER'] = offensive_stats['PLAYER'].apply(normalize_name)
  defensive_stats = get_defensive_stats(end_season)
  defensive_stats['PLAYER'] = defensive_stats['PLAYER'].apply(normalize_name)
  advanced_stats = get_advanced_stats(end_season)
  advanced_stats['PLAYER'] = advanced_stats['PLAYER'].apply(normalize_name)
  team_record = get_team_record(season)
  merge_first = player_stats.merge(offensive_stats, on = 'PLAYER', how = 'left')
  merge_second = merge_first.merge(defensive_stats, on = 'PLAYER', how = 'left')
  merge_third = merge_second.merge(advanced_stats, on = 'PLAYER', how = 'left')
  final_data = merge_third.merge(team_record, left_on = 'TEAM_ID', right_on = 'TeamID', how = 'left')
  final_data = final_data.drop(columns = ['TeamID'])
  final_data['All_NBA'] = final_data['AWARDS'].apply(is_all_nba).astype('Int64')
  final_data = final_data.drop(columns = ['AWARDS'])
  final_data['PLAYER'] = final_data['PLAYER'].apply(capitalize_name)
  return final_data

In [ ]:
create_final_data('2020-21')

In [ ]:
for i in range(0, 5) :
  data = create_final_data('202' + str(i) + '-2' + str(i+1))
  data.to_excel('data_202' + str(i+1) + '.xlsx', index = False)

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

def preprocess(df):
    df = df.copy()
    if 'Record' in df.columns:
      df[['WINS', 'LOSSES']] = df['Record'].str.split('-', expand=True).astype(float)
      df['TEAM_W_PCT'] = (df['WINS'] / (df['WINS'] + df['LOSSES'])).round(3)
      drop_cols = ['WINS', 'LOSSES', 'Record']
      df.drop(columns=[col for col in drop_cols if col in df.columns], inplace=True)
    df.fillna(0, inplace=True)
    return df

data_2021 = pd.read_excel('data_2021.xlsx')
data_2021['All_NBA'] = data_2021['All_NBA'].fillna(0)
data_2021['All_NBA'] = data_2021['All_NBA'].astype(int)
data_2021 = preprocess(data_2021)
data_2022 = pd.read_excel('data_2022.xlsx')
data_2022['All_NBA'] = data_2022['All_NBA'].fillna(0)
data_2022['All_NBA'] = data_2022['All_NBA'].astype(int)
data_2022 = preprocess(data_2022)
data_2023 = pd.read_excel('data_2023.xlsx')
data_2023['All_NBA'] = data_2023['All_NBA'].fillna(0)
data_2023['All_NBA'] = data_2023['All_NBA'].astype(int)
data_2023 = preprocess(data_2023)
data_2024 = pd.read_excel('data_2024.xlsx')
data_2024['All_NBA'] = data_2024['All_NBA'].fillna(0)
data_2024['All_NBA'] = data_2024['All_NBA'].astype(int)
data_2024 = preprocess(data_2024)
data_2025 = pd.read_excel('data_2025.xlsx')
data_2025 = preprocess(data_2025)
data_2025 = data_2025[data_2025['GP'] > 64]
train_data = pd.concat([data_2021, data_2022, data_2023, data_2024])
train_data = train_data[train_data['GP'] > 64]

label = 'All_NBA'
other = ['PLAYER_ID', 'PLAYER', 'TEAM_ID', 'TeamName']
features = [col for col in train_data.columns if col not in [label] + other]

X = train_data[features]
y = train_data[label]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
model = XGBClassifier(
    booster = 'gblinear',
    objective='multi:softprob',
    num_class=4,
    learning_rate=0.05,
    n_estimators=1000,
    reg_alpha=0,
    reg_lambda=1
)
model.fit(X_scaled, y)
X_2025 = data_2025[features]
X_2025_scale = scaler.transform(X_2025)
probabilities = model.predict_proba(X_2025_scale)
prob = probabilities[:, 1:]
total = prob.sum(axis = 1)
data_2025['All_NBA_Probability'] = total
sorted_players = data_2025.sort_values(by='All_NBA_Probability', ascending=False).head(15)

teams = {1 : [], 2 : [], 3 : []}
for i, (_, row) in enumerate(sorted_players.iterrows()):
    if i < 5:
        teams[1].append(row['PLAYER'])
    elif i < 10:
        teams[2].append(row['PLAYER'])
    else:
        teams[3].append(row['PLAYER'])
data_2025['All_NBA'] = 0
for team, players in teams.items():
    for player in players:
        data_2025.loc[data_2025['PLAYER'] == player, 'All_NBA'] = team
data_2021.to_excel('data_2021.xlsx', index = False)
data_2022.to_excel('data_2022.xlsx', index = False)
data_2023.to_excel('data_2023.xlsx', index = False)
data_2024.to_excel('data_2024.xlsx', index = False)
data_2025.to_excel('data_2025.xlsx', index = False)
# 🖨️ Show results
print("🔵 All-NBA First Team 2025:")
print(teams[1])
print("\n🟢 All-NBA Second Team 2025:")
print(teams[2])
print("\n🟠 All-NBA Third Team 2025:")
print(teams[3])
dump = model.get_booster().get_dump(with_stats=True)

# Print the first 20 lines of class 0's dump
print("===== Class 0 Dump Preview =====")
lines = dump[0].split('\n')
weights = [float(value) for value in lines[6:-1]]
avg_weights = [sum(weights[i + j*21] for j in range(4)) / 4
    for i in range(21)]
avg_weights
feature_weights = pd.DataFrame({
    'Feature': features,
    'Average Weight': avg_weights
})
feature_weights.sort_values(by='Average Weight', ascending=False)
feature_weights.to_excel('feature_weights.xlsx', index = False)

🔵 All-NBA First Team 2025:
['Nikola Jokic', 'Shai Gilgeousalexander', 'Giannis Antetokounmpo', 'Jayson Tatum', 'Lebron James']

🟢 All-NBA Second Team 2025:
['James Harden', 'Anthony Edwards', 'Cade Cunningham', 'Stephen Curry', 'Karlanthony Towns']

🟠 All-NBA Third Team 2025:
['Domantas Sabonis', 'Tyrese Haliburton', 'Donovan Mitchell', 'Jalen Brunson', 'Jalen Williams']
===== Class 0 Dump Preview =====
